In [2]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import datetime
pd.set_option("display.max_rows", None, "display.max_columns", None)
oct_avl_raw = pd.read_csv('October_avl_data.csv') #2.8GB file directly from Xiaotong

In [4]:
oct_avl_raw.columns

Index(['bus_state_id', 'event_time', 'event_type', 'bus_id', 'route_id',
       'trip_id', 'stop_id', 'stop_sequence', 'odometer_distance',
       'dwell_time', 'passenger_load', 'ron', 'fon', 'roff', 'foff', 'day'],
      dtype='object')

In [7]:
'''
Xiaotong's cleaning code
'''

def CTA_avl_data_cleaning(pd_avl_raw_data):
    """
    Function to clean avl data
    :param pd_avl_raw_data: raw avl data in pandas dataframe format
    :return: clean avl data
    """

    pd_avl_raw_data['day'] = pd.to_datetime(pd_avl_raw_data['event_time']).dt.day
    avl_raw_data = pd_avl_raw_data.sort_values(by=['route_id', 'trip_id', 'day', 'stop_sequence', 'event_time']).values.tolist()

    # Combine ron&fon and roff & foff
    raw_data_ = []
    for i in avl_raw_data:
        pax_off = i[-2] + i[-3]
        pax_on = i[-4] + i[-5]
        new_row = i[:-5] + [pax_on, pax_off]
        raw_data_.append(new_row)
   
    
    # Drop data without stop_id, trip_id, bus_id.
    # Drop stop event type 5
    data = []
    for i in raw_data_:
        try:
            i[1].day
        except AttributeError:
            i[1] = pd.Timestamp(i[1])

        if np.isnan(i[3]):
            continue
        elif np.isnan(i[5]):
            continue
        elif np.isnan(i[6]):
            continue
        elif i[2] == 3 or i[2] == 4:
            new_row = [int(i[0]), pd.Timestamp(i[1]), int(i[2]), int(i[3]), str(i[4])] \
                      + [str(int(i[5])) + "-" + str(i[1])[:10], int(i[6])] + i[7:]
            data.append(new_row)

    # If two rows represent the stop event at the same bus stop, keep the first one.
    new_data = []
    reversed_data = list(reversed(data))
    for i in range(len(reversed_data) - 1):
        if reversed_data[i][6] == reversed_data[i + 1][6] and reversed_data[i][5] == reversed_data[i + 1][5]:
            continue
        new_data.append(reversed_data[i])
    new_data.append(reversed_data[-1])
    new_data = list(reversed(new_data))

    # Processed passenger loading with raw passenger_on and passenger_off
    processed_data = []
    for i in range(len(new_data)):
        record = new_data[i]
        if i == 0:
            pax_on = record[11]
            loading = pax_on
            new_record = record[:10] + [loading] + record[11:]
            processed_data.append(new_record)
        elif i == len(new_data) - 1:
            loading = 0
            new_record = record[:10] + [loading] + record[11:]
            processed_data.append(new_record)
        else:
            prev_record = new_data[i - 1]
            next_record = new_data[i + 1]
            pax_on = record[11]
            pax_off = record[12]
            if record[7] == 1 or prev_record[5] != record[5]:  # first trip record
                loading = pax_on
                new_record = record[:10] + [loading] + record[11:]
                processed_data.append(new_record)
            elif record[5] == next_record[5]:  # not first trip & same trip with the next record
                prev_loading = processed_data[i - 1][10]
                loading = max(0, prev_loading + pax_on - pax_off)
                new_record = record[:10] + [loading] + record[11:]
                processed_data.append(new_record)
            else:  # not first trip & not same trip with next record
                loading = 0
                new_record = record[:10] + [loading] + record[11:]
                processed_data.append(new_record)

    return processed_data

In [8]:
'''
Pass all AVL raw data through Xiaotong's cleaning code
Save as csv
'''
xiaotong_avl = CTA_avl_data_cleaning(oct_avl_raw)
avl = pd.DataFrame(xiaotong_avl)
avl.to_csv("xiaotong_avl.csv", index=False)

raw_data first row [17418448814, '2017-10-09 13:16:01', 5, 8148, 0, 119444752.0, nan, -1, 0, 144, 0, 0, 0]


NameError: name 'np' is not defined

In [2]:
xiaotong_avl = pd.read_csv("xiaotong_avl.csv") #1.37 GB

#saving as csv casts type back to string
xiaotong_avl['day'] = pd.to_datetime(xiaotong_avl['1']).dt.day

/Users/tingli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [87]:
print(xiaotong_avl.shape) #15,778,236
display(xiaotong_avl.head(10))

(15778236, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,day
0,17382022573,2017-10-02 17:18:18,3,7995,3,119546472-2017-10-02,14799,1,404557,573,1,1,0,2
1,17382022575,2017-10-02 17:20:47,3,7995,3,119546472-2017-10-02,593,2,404993,10,2,1,0,2
2,17382022579,2017-10-02 17:22:02,4,7995,3,119546472-2017-10-02,595,3,405584,0,2,0,0,2
3,17382022585,2017-10-02 17:23:52,4,7995,3,119546472-2017-10-02,597,4,406352,0,2,0,0,2
4,17382022588,2017-10-02 17:26:52,4,7995,3,119546472-2017-10-02,14488,5,407408,0,2,0,0,2
5,17382022592,2017-10-02 17:27:19,4,7995,3,119546472-2017-10-02,1100,6,407977,0,2,0,0,2
6,17382022593,2017-10-02 17:28:44,3,7995,3,119546472-2017-10-02,1101,7,408487,6,2,0,0,2
7,17382022596,2017-10-02 17:29:22,3,7995,3,119546472-2017-10-02,1102,8,409019,3,2,0,0,2
8,17382022600,2017-10-02 17:30:34,3,7995,3,119546472-2017-10-02,1103,9,409451,13,5,3,0,2
9,17382022605,2017-10-02 17:32:17,3,7995,3,119546472-2017-10-02,1104,10,410563,4,6,1,0,2


In [18]:
'''
Extract all rows where date is October 19th
'''
oct_19 = xiaotong_avl[xiaotong_avl['day']==19]   #563,267 rows

In [43]:
'''
Split tripid and date string column, create new column 'trip_id'
Save to csv
'''
tripid_date = oct_19['5'].str.partition("-", True)
oct_19['trip_id'] = tripid_date[0]

#len(oct19['trip_id'].unique()) #2941

oct_19_avl_trips = oct_19.drop_duplicates(subset=['trip_id', '4']) #drop rows with duplicate trip and route_ids
oct_19_avl_trips.to_csv('xiaotong_oct19.csv', index=False) #409 kB

In [44]:
'''
Read October 19 csv, corresponding gtfs trips.txt from CTA website
'''
oct_19 = pd.read_csv('xiaotong_oct19.csv')
gtfs_trips = pd.read_csv("gtfs_trips_oct_17.txt", delimiter=",", 
                usecols=['route_id', "service_id","trip_id", "schd_trip_id"]) #90,230 rows


/Users/tingli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
gtfs_trips.head(10)

,route_id,service_id,trip_id,schd_trip_id
0,1,48201,482119547274,119547274
1,1,48201,482119547276,119547276
2,1,48201,482119547277,119547277
3,1,48201,482119547278,119547278
4,1,48201,482119547279,119547279
5,1,48201,482119547280,119547280
6,1,48201,482119547281,119547281
7,1,48201,482119547282,119547282
8,1,48201,482119547283,119547283
9,1,48201,482119547287,119547287


In [45]:
print(len(oct_19['4'].unique())) #where column 4 is route_id
print(len(gtfs_trips['route_id'].unique()))

128
135


In [47]:
'''
Checks and keep only when AVL trip_id exists in GTFS trip_id
avl_not_in_gtfs keeps track of AVL trip_ids not in GTFS trip_ids, for pattern recognition later on
'''
avl_trip_df = {'route_id':[], 'service_id':[], 'trip_id':[]}
avl_not_in_gtfs_count = 0
avl_not_in_gtfs = []

for i, row in tqdm(oct_19.iterrows(), total=oct_19.shape[0]):
    
    matches = gtfs_trips.loc[(gtfs_trips['schd_trip_id']==int(row['trip_id']))]
    
    if not matches.empty:
        for a, r in matches.iterrows():
            avl_trip_df['route_id'].append(r['route_id'])
            avl_trip_df['service_id'].append(r['service_id'])
            avl_trip_df['trip_id'].append(r['trip_id'])
    else: 
        avl_not_in_gtfs_count += 1
        avl_not_in_gtfs.append(row)
        
avl_not_in_gtfs = pd.DataFrame(avl_not_in_gtfs)

100%|██████████| 4108/4108 [00:16<00:00, 253.39it/s]


In [70]:
'''
Info about avl trips not found in gtfs
'''
#print("Total unique AVL trip_ids for 10/19/2017:", oct_19.shape[0])
#print("Number of AVL trips not found in GTFS:", avl_not_in_gtfs_count)
display(avl_not_in_gtfs)



,0,1,2,3,4,5,6,7,8,9,10,11,12,day,trip_id
28,17463135738,2017-10-19 00:14:31,3,1149,5,119619592-2017-10-19,10178,1,101612,183,14,14,0,19,119619592
29,17462457650,2017-10-19 01:12:12,3,1121,5,119619600-2017-10-19,10178,1,644740,459,39,39,0,19,119619600
30,17464809338,2017-10-19 07:39:25,3,1324,8,119547360-2017-10-19,3681,1,5296,9,1,1,0,19,119547360
31,17466336302,2017-10-19 09:23:16,3,7936,8,119547368-2017-10-19,3681,1,160322,1001,6,6,0,19,119547368
32,17467128092,2017-10-19 10:36:54,3,1252,8,119547376-2017-10-19,3681,1,306308,1358,5,5,0,19,119547376
33,17466022226,2017-10-19 12:04:38,3,1193,8,119547384-2017-10-19,3681,1,460870,886,4,4,0,19,119547384
34,17467834039,2017-10-19 13:12:48,3,1295,8,119547392-2017-10-19,3681,1,461407,932,8,8,0,19,119547392
35,17462718602,2017-10-19 14:38:45,3,1259,8,119547400-2017-10-19,3681,1,305257,899,7,7,0,19,119547400
36,17466744585,2017-10-19 15:56:45,3,7996,8,119547408-2017-10-19,3681,1,608988,841,1,1,0,19,119547408
37,17467129597,2017-10-19 17:59:27,3,1252,8,119547416-2017-10-19,3681,1,606002,84,2,2,0,19,119547416


In [88]:
'''
I don't know what exactly xiaotong's code generates in terms of column names
My guess is that column 4 is route_id, column 5 is concatenated trip_id and date, 
    column 7 is service_id.
'''
print(avl_not_in_gtfs['7'].unique())
print(avl_not_in_gtfs['trip_id'].nunique())

print(oct_19['7'].unique())
print(oct_19['trip_id'].nunique())

print(gtfs_trips['service_id'].unique())

[ 1  8 31  2  3 13 48 24]
1116
[ 2  1  8 31  3 19  5 13 48 15 25  7 20 40 24]
2941
[ 48201  48301  48206  48209  48224  48226  48306  48309  48324  48326
  48220  48222  48320  48322  48202  48205  48302  48305  48204  48207
  48208  48212  48215  48304  48307  48308  48312  48315  13001  13006
  13009  13011  13016  13019  48211  48311   1901   1902   1903   1911
  48213  48313  48203  48303  48216  48218  48316  48318 105401 105406
 105408 105409 105412 105415 105407 205401]


In [86]:
'''
Drop duplicate entries and save to txt file
'''
dfObj = pd.DataFrame(avl_trip_df) #5,131 rows
duplicateRowsDF = dfObj[dfObj.duplicated()]
#print(duplicateRowsDF) #1,489 rows

avl_trips = dfObj.drop_duplicates()
#print(avl_trips.shape) #3642 rows

avl_trips.to_csv("trips.txt", index=False, header=True) 

## Get rid of invalid service ids for 10/19/17

In [15]:
gtfs_path = 'partridge_data/gtfs-10-22-17/' #encompasses 10-19-17 as well
#avl_path = 'partridge_data/avl-10-19/'

calendar = pd.read_csv(gtfs_path+"calendar.txt", delimiter=",")
calendar_dates = pd.read_csv(gtfs_path+"calendar_dates.txt", delimiter=",")
gtfs_trips = pd.read_csv(gtfs_path+"trips.txt", delimiter=",")
avl_trips = pd.read_csv("trips.txt", delimiter=",")

gtfs_trips['service_id'] = gtfs_trips['service_id'].astype('int')
avl_trips['service_id'] = avl_trips['service_id'].astype('int')

/Users/tingli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
# GET VALID SERVICE IDs for 10/19/2017 (thursday)
oct19 = 20171019

valid_ids = calendar.loc[(calendar['thursday']==1) &
                         (calendar['start_date'] <= oct19) &
                         (calendar['end_date'] >= oct19)]
serviceIDs_10_19 = set(valid_ids['service_id'])

In [17]:
print(serviceIDs_10_19)

{105401, 105402, 105403, 105404, 105410, 105411, 105412, 48201, 13001, 48202, 48203, 48204, 105418, 105419, 105422, 105423, 48210, 48211, 48212, 105424, 105425, 105430, 105431, 205401, 48218, 48219, 105434, 105435, 48222, 48223, 48224, 48225, 105436, 105437, 105438, 105439, 48230, 48231, 105440, 48234, 48235, 48236, 48237, 48238, 48239, 48240}


In [18]:
'''
Exceptions in CALENDAR DATES
1 - Service has been added for the specified date.
2 - Service has been removed for the specified date.
'''
exception1 = calendar_dates.loc[(calendar_dates['date'] == oct19) &
                                (calendar_dates['exception_type'] == 1)]
exception2 = calendar_dates.loc[(calendar_dates['date'] == oct19) &
                                (calendar_dates['exception_type'] == 2)]
#both empty
exception1 = set(exception1['service_id'])
exception2 = set(exception2['service_id'])

valid_services = serviceIDs_10_19 - exception2
valid_services = valid_services.union(exception1)

In [19]:
'''
Filter all trips with service_id in valid_services
'''
valid_gtfs_trips = gtfs_trips.loc[gtfs_trips['service_id'].isin(valid_services)]

print(valid_gtfs_trips['trip_id'].nunique()) #20582
print(avl_trips.shape[0]) #3642

20582
3642


In [20]:
print(avl_trips['service_id'].unique())

[48201 48301 48204 48304 48209 48309 48222 48322 48215 48315 48212 48312
 48211 48311 48202 48302 48216 48316 48205 48305]
